# Hillis-Steele scan 

## To determine the prefix-sum of an array using parallel implementation of Hillis-Steele scan(inclusive scan)

In [1]:
using Pkg
Pkg.add("CuArrays")
Pkg.add("CUDAnative")
Pkg.add("CUDAdrv")
Pkg.add("StaticArrays")
Pkg.add("BenchmarkTools")
Pkg.add("Test")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [be33ccc6] + CUDAnative v2.2.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [c5f51814] + CUDAdrv v3.0.1
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
  [90137ffa] + StaticArrays v0.11.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
 Installed BenchmarkTools ─ v0.4.2
  Updating `~/.julia/environments/v1.0/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
 

In [0]:
using CUDAnative, CUDAdrv, CuArrays, StaticArrays, BenchmarkTools, Test

In [3]:
const N = 2^15
const threads = 2^9
input = rand(N)

# Number of elements to be handled by one thread
const chunk = Int(ceil(N/threads))

64

In [4]:
# Parallel implementation of scan using thread overloading

function scan(d_input)

  idx=(blockIdx().x - 1) * blockDim().x + threadIdx().x
    
  temp = @MArray zeros(chunk)
  
  start = chunk * idx - (chunk-1)
  stop = chunk * idx
  
  # Bounds check
  if(idx <= min(size(d_input,1),blockDim().x))
    step = 1
    while step < size(d_input,1)
    
    for i=start:stop
      if(i <= size(d_input,1))
        temp[i - (idx-1)*chunk] = d_input[i]
      end
    end
        
    for i=start:stop
      if(i + step <= size(d_input,1))
        temp[i - (idx-1)*chunk] += d_input[i + step]
      end
    end
    
    sync_threads()
    
    for i=start:stop
      if(i + step <= size(d_input,1))
        d_input[i + step] = temp[i - (idx-1)*chunk]
      end
    end
    
    step *= 2
    
    end
  end

return 
end

scan (generic function with 1 method)

In [0]:
d_input = CuArray(input)
@cuda threads = threads scan(d_input)

In [6]:
h_output = Array(d_input)

32768-element Array{Float64,1}:
     0.8480796367787318
     1.2895616854823961
     1.4318049501822283
     1.6638508473907994
     2.108067134054021 
     3.043014773082673 
     3.402319783116667 
     3.4788847696534653
     4.0130649172302615
     4.445416591435246 
     4.6474976689544505
     5.621696792188163 
     6.616240449813281 
     ⋮                 
 16339.855034690438    
 16339.402569634427    
 16340.389857818265    
 16342.874609959836    
 16343.675543041132    
 16344.222502458993    
 16343.188136068726    
 16345.509241890586    
 16344.499983811376    
 16344.456185058487    
 16345.45121926814     
 16347.447718011867    

## Output is the prefix-sum (inclusive scan) of the input array 